In [220]:
import numpy as np
import pandas as pd
from  matplotlib import pyplot as plt
import seaborn as sns

In [221]:
df_train=pd.read_csv('Black_Friday_Sales_train.csv')

In [222]:
df_test=pd.read_csv('Black_Friday_Sales_test.csv')

In [223]:
df_test['source']='Test'
df_train['source']='Train'

In [224]:
df=pd.concat([df_train,df_test])

In [225]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
User_ID,783667.0,NaN,NaN,NaN,1003028.995753,1727.266668,1000001.0,1001519.0,1003075.0,1004478.0,1006040.0
Product_ID,783667,3677,P00265242,2709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,783667,2,M,590031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,783667,7,26-35,313015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,783667.0,NaN,NaN,NaN,8.0793,6.522206,0.0,2.0,7.0,14.0,20.0
City_Category,783667,3,B,329739,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stay_In_Current_City_Years,783667,5,1,276425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Marital_Status,783667.0,NaN,NaN,NaN,0.409777,0.491793,0.0,0.0,0.0,1.0,1.0
Product_Category_1,783667.0,NaN,NaN,NaN,5.366196,3.87816,1.0,1.0,5.0,8.0,20.0
Product_Category_2,537685.0,NaN,NaN,NaN,9.844506,5.089093,2.0,5.0,9.0,15.0,18.0


      Inference from data :
              1. User_ID = can be removed - should not have any impact on the outcome
              2. ProductID = Change into prod identifier to reduce the number of unique values
              3. Gender = Catergorical data M/F so can be encoded
              4. Age = It is range value not sure what to do for now. Can i substute numbers for age ran - ordinal encoding
              5. Occupation = Masked values not sure what to do
              6. City_Category = Only 3 unique values and no null - can be encoded
              7. Stay_In_Current_City_Years - has one value that is 4+ can be converted into int and made 1-4
              8. Marital_Status = Aleady encoded to 0 and 1 no changes required
              9. Product_Category_1 - Looks good not skewed
              10. Product_Category_2 - Need to fill NaN values
              11. Product_Category_3 - Need to fill Nan values
              

In [226]:
df['Product_ID'].str[0:4].value_counts()
df['Product_identifier']=df['Product_ID'].str[0:4]

In [227]:
df['Stay_In_Current_City_Years'].replace({'1':1,'0':0,'2':2,'3':3,'4+':4},inplace=True)

In [228]:
df['Age'].replace({'0-17':0,'18-25':1,'26-35':2,'36-45':3,'46-50':4,'51-55':5,'55+':6},inplace=True)

In [229]:
df['Product_Category_2'].value_counts()

8.0     91317
14.0    78834
2.0     70498
16.0    61687
15.0    54114
5.0     37165
4.0     36705
6.0     23575
11.0    20230
17.0    19104
13.0    15054
9.0      8177
12.0     7801
10.0     4420
3.0      4123
18.0     4027
7.0       854
Name: Product_Category_2, dtype: int64

In [230]:
df['Product_Category_2'] = df.groupby(['Product_identifier']).Product_Category_2.transform(lambda x: x.fillna(x.mean()))

## After the substution of values to Product_Category_2  I am comparing mean before and after -to confirm that there is not much deviation and it remans almost same 

In [231]:
df['Product_Category_2'].mean()

9.879140378712298

In [232]:
df['Product_Category_2'].isnull().sum()

0

In [233]:
df['Product_Category_3'] = df.groupby(['Product_identifier']).Product_Category_3.transform(lambda x: x.fillna(x.mean()))

In [234]:
df['Product_Category_3'].mean()

12.763426903056148

In [237]:
dummies=pd.get_dummies(df['Gender'])
dummies1=pd.get_dummies(df['City_Category'])
df=pd.concat([df,dummies,dummies1],axis=1)
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,Product_identifier,F,M,A,B,C
0,1000001,P00069042,F,0,10,A,2,0,3,10.023745,12.758483,8370.0,Train,P000,1,0,1,0,0
1,1000001,P00248942,F,0,10,A,2,0,1,6.000000,14.000000,15200.0,Train,P002,1,0,1,0,0
2,1000001,P00087842,F,0,10,A,2,0,12,10.023745,12.758483,1422.0,Train,P000,1,0,1,0,0
3,1000001,P00085442,F,0,10,A,2,0,12,14.000000,12.758483,1057.0,Train,P000,1,0,1,0,0
4,1000002,P00285442,M,6,16,C,4,0,8,9.995579,12.508731,7969.0,Train,P002,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036,P00118942,F,2,15,B,4,1,8,9.425940,12.481481,NaN,Test,P001,1,0,0,1,0
233595,1006036,P00254642,F,2,15,B,4,1,5,8.000000,12.508731,NaN,Test,P002,1,0,0,1,0
233596,1006036,P00031842,F,2,15,B,4,1,1,5.000000,12.000000,NaN,Test,P000,1,0,0,1,0
233597,1006037,P00124742,F,4,1,C,4,0,10,16.000000,12.481481,NaN,Test,P001,1,0,0,0,1


In [238]:
df=df.drop(['Gender','City_Category'],axis=1)

In [239]:
df

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,source,Product_identifier,F,M,A,B,C
0,1000001,P00069042,0,10,2,0,3,10.023745,12.758483,8370.0,Train,P000,1,0,1,0,0
1,1000001,P00248942,0,10,2,0,1,6.000000,14.000000,15200.0,Train,P002,1,0,1,0,0
2,1000001,P00087842,0,10,2,0,12,10.023745,12.758483,1422.0,Train,P000,1,0,1,0,0
3,1000001,P00085442,0,10,2,0,12,14.000000,12.758483,1057.0,Train,P000,1,0,1,0,0
4,1000002,P00285442,6,16,4,0,8,9.995579,12.508731,7969.0,Train,P002,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036,P00118942,2,15,4,1,8,9.425940,12.481481,NaN,Test,P001,1,0,0,1,0
233595,1006036,P00254642,2,15,4,1,5,8.000000,12.508731,NaN,Test,P002,1,0,0,1,0
233596,1006036,P00031842,2,15,4,1,1,5.000000,12.000000,NaN,Test,P000,1,0,0,1,0
233597,1006037,P00124742,4,1,4,0,10,16.000000,12.481481,NaN,Test,P001,1,0,0,0,1


In [242]:
train_preprocessed = df[df['source']=='Train'].drop(columns=['source','Product_identifier'])
test_preprocessed = df[df['source']=='Test'].drop(columns=['source', 'Product_identifier'])

In [243]:
train_preprocessed

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,F,M,A,B,C
0,1000001,P00069042,0,10,2,0,3,10.023745,12.758483,8370.0,1,0,1,0,0
1,1000001,P00248942,0,10,2,0,1,6.000000,14.000000,15200.0,1,0,1,0,0
2,1000001,P00087842,0,10,2,0,12,10.023745,12.758483,1422.0,1,0,1,0,0
3,1000001,P00085442,0,10,2,0,12,14.000000,12.758483,1057.0,1,0,1,0,0
4,1000002,P00285442,6,16,4,0,8,9.995579,12.508731,7969.0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550063,1006033,P00372445,5,13,1,1,20,10.717099,14.027454,368.0,0,1,0,1,0
550064,1006035,P00375436,2,1,3,0,20,10.717099,14.027454,371.0,1,0,0,0,1
550065,1006036,P00375436,2,15,4,1,20,10.717099,14.027454,137.0,1,0,0,1,0
550066,1006038,P00375436,6,1,2,0,20,10.717099,14.027454,365.0,1,0,0,0,1


In [244]:
test_preprocessed

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,F,M,A,B,C
0,1000004,P00128942,4,7,2,1,1,11.00000,12.481481,NaN,0,1,0,1,0
1,1000009,P00113442,2,17,0,0,3,5.00000,12.481481,NaN,0,1,0,0,1
2,1000010,P00288442,3,1,4,1,5,14.00000,12.508731,NaN,1,0,0,1,0
3,1000010,P00145342,3,1,4,1,4,9.00000,12.481481,NaN,1,0,0,1,0
4,1000011,P00053842,2,1,1,0,4,5.00000,12.000000,NaN,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036,P00118942,2,15,4,1,8,9.42594,12.481481,NaN,1,0,0,1,0
233595,1006036,P00254642,2,15,4,1,5,8.00000,12.508731,NaN,1,0,0,1,0
233596,1006036,P00031842,2,15,4,1,1,5.00000,12.000000,NaN,1,0,0,1,0
233597,1006037,P00124742,4,1,4,0,10,16.00000,12.481481,NaN,1,0,0,0,1


In [245]:
train_preprocessed.to_csv("Black_Friday_train_preprocessed.csv", index = False)
test_preprocessed.to_csv("Black_Friday_test_preprocessed.csv", index = False)